# imports

In [27]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [28]:
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
from utils import plot_image
from datetime import datetime
from dateutil.relativedelta import relativedelta
import glob


In [29]:
from sentinelhub import SHConfig
config = SHConfig()

In [30]:
#create an client and instance on sentinal-hub
config.sh_client_id = '9d0e7fa9-5e61-4762-bdee-e17f312c8eb5'
config.sh_client_secret = 'Qhh(WQu@wuP&(!uuWtQ@Uw^9o?/1wLRm%4yvufg9'
config.instance_id = "a5f10728-b270-4276-a238-9258e142713a"
config.save()

In [31]:
if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")

In [32]:
from sentinelhub import (
    CRS,
    BBox,
    Geometry,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# Setting area of interest

In [33]:
poly = [[80.065957,8.68801],[80.376549,8.515564],[80.032973,8.504699],[80.065957,8.68801]]
import numpy as np


In [34]:
def bbox_cord(poly):
    poly_arr = np.array(poly)
    df = pd.DataFrame(poly_arr, columns = ['lat','long'])
    Xmin = df['lat'].min()
    Ymin = df['long'].min()
    Xmax = df['lat'].max()
    Ymax = df['long'].max()
    bbox = [Xmin,Ymin,Xmax,Ymax]
    return bbox

In [35]:
bbox_generated = bbox_cord(poly)
bbox_generated

[80.032973, 8.504699, 80.376549, 8.68801]

In [36]:
final_bbox = BBox(bbox=bbox_generated, crs=CRS.WGS84)
geometry = Geometry(geometry={"type":"Polygon","coordinates":[poly]}, crs=CRS.WGS84)


In [37]:
resolution =20
box_size = bbox_to_dimensions(final_bbox, resolution= resolution)
print(f"Image shape at {resolution} m resolution: {box_size} pixels")

Image shape at 20 m resolution: (1893, 1009) pixels


In [38]:
#date range
current_date = datetime.today().strftime('%Y-%m-%d')
past_date = datetime.today() - relativedelta(months=10)
past_date = past_date.strftime('%Y-%m-%d')

In [39]:
evalscript_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B08","B02","B03","B04"],
                units: "DN"
            }],
            output: {
                bands: 4,
                sampleType: "float32" 
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B08,
                sample.B02,
                sample.B03,
                sample.B04,
];
    }
"""
#32 bit Tiff output wuth 4 bands
request_all_bands = SentinelHubRequest(
    data_folder="Data", #data store directory
    evalscript=evalscript_all_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=( past_date,current_date),
            mosaicking_order=MosaickingOrder.LEAST_CC,
            other_args={"dataFilter": {"maxCloudCoverage": 20}},
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=final_bbox,
    size=box_size,
    geometry = geometry,
    config=config,
)

In [26]:
#API request
all_bands_response = request_all_bands.get_data()

DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/process
with HTTPError:
403 Client Error: Forbidden for url: https://services.sentinel-hub.com/api/v1/process
Server response: "{"status": 403, "reason": "Forbidden", "message": "Invalid or expired account.", "code": "COMMON_INSUFFICIENT_PERMISSIONS"}"

In [ ]:
#visualize RGB
plot_image(all_bands_response[0][:, :, [1, 2, 3]], factor=3.5 / 1e4, clip_range=(0, 1))

In [18]:
all_bands_img = request_all_bands.get_data(save_data=True)

rename the file

In [10]:
list_of_files = glob.glob(r'C:\Users\Harith\EngenuityAI\SPE\SPE testing\SPE-testing\Sentinel-hub API\Data\*') # * means all if need specific format then *.csv

In [11]:
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

C:\Users\Harith\EngenuityAI\SPE\SPE testing\SPE-testing\Sentinel-hub API\Data\05fcfd50ddb933d26862424c909c4870


In [12]:
new_name = r'C:\Users\Harith\EngenuityAI\SPE\SPE testing\SPE-testing\Sentinel-hub API\Data\triangle'
os.rename(latest_file, new_name)